# Computing the bicrystallographic aspects of Misorientations (FCC lattice)

In [1]:
import byxtal.lattice as gbl;
import byxtal.csl_utility_functions as cuf;
import byxtal.find_csl_dsc as fcd;
import numpy as np;
import byxtal.tools as bxt;
import byxtal.misorient_fz as mfz;
import byxtal.disorient_symm_props as dsp;
from sympy.matrices import Matrix, eye, zeros;

Define the lattice: Here we use the `lattice` module and use the default option. The default option will initialize a cubic lattice with $a = 1$.

In [2]:
l1 = gbl.Lattice()
sig_type = 'common'
l_p_po = Matrix(l1.l_p_po)

First, initialize the $\Sigma$ number of interest. Note that for cubic crystals, CSL rotations exist only for odd-values of $\Sigma$.

In [14]:
sig_num = 651

The function `csl_rotations` in the module `byxtal.csl_utility_functions`, will compute the $\Sigma$ rotation (a misorientation) in the reference frame of the primitive cell.

In [15]:
s1 = cuf.csl_rotations(sig_num, sig_type, l1);
print(s1)

{'N': array([[[ 645.,  -54.,   54.],
        [  74.,  666.,  -15.],
        [ -70.,   21.,  630.]],

       [[ 515., -265.,  149.],
        [ 365.,  716.,   74.],
        [-264.,   60.,  519.]],

       [[ 449., -346.,   44.],
        [ 375.,  681.,   15.],
        [-124.,  155.,  620.]],

       [[ 549., -225.,  141.],
        [ 315.,  714.,   56.],
        [-240.,   45.,  536.]],

       [[ 570., -195.,  174.],
        [ 250.,  634.,  -95.],
        [-161.,  175.,  595.]],

       [[ 495., -286.,  156.],
        [ 420.,  770.,  231.],
        [-396.,  -75.,  396.]],

       [[ 474., -300.,  191.],
        [ 426.,  711.,   92.],
        [-315.,   84.,  476.]],

       [[ 405., -390.,   54.],
        [ 474.,  725.,  150.],
        [-249.,   71.,  531.]],

       [[ 386., -408.,   51.],
        [ 434.,  651.,    0.],
        [-115.,  216.,  624.]],

       [[ 465., -310.,  186.],
        [ 390.,  629.,  -54.],
        [-189.,  231.,  567.]],

       [[ 399., -392.,   84.],
        [ 450

As can be obsrved from the output above, there exist multiple rotations that correspond to a given $\Sigma$ number. This is usually the case when $\Sigma$ is large. In the following, we will compute the bycrystallography properties for one of the misorientations (e.g. corresponding to the second rotation in the array). The bycrystallography properties of interest are as follows:
+ The Coincidence Site Lattice (CSL). For more details, refer to this [tutorial](https://www.tf.uni-kiel.de/matwis/amat/def_en/kap_7/backbone/r7_1_2.html).
+ The DSC lattice that defines the **D**isplacements that are **S**ymmetry **C**onserving (also referred to as the **D**isplacement **S**hift **C**omplete). For more details, refer to this [tutorial](https://www.tf.uni-kiel.de/matwis/amat/def_en/kap_7/backbone/r7_1_3.html)
+ The boundary-plane symmetry group, and the symmetry axes. For more details refer to the following article:
    - [**Symmetries in the representation of grain boundary-plane distributions**](https://www.tandfonline.com/doi/full/10.1080/14786435.2012.722700?casa_token=4nDY3LLK9vQAAAAA%3AgECePf6JCdOPx6K_7cfxrJATdUAwMOSe3-F5NIdpjuI26uAPQfkkSTa61np1AWoDGQ3wcJrh3SS9). Patala, S., & Schuh, C. A. (2013). . Philosophical Magazine, 93(5), 524-573.

In [17]:
ct1 = 7
sig_id = str(sig_num)+'_'+str(ct1)
print(sig_id)

651_7


The $\Sigma$-misorientation is a rotation operation that is defined in the $P$ lattice reference frame and is computed as shown in the code snippet below. We use the variable `T_p1top2_p1` denote the **T**ransformation (`T`) from the $P1$ lattice to the $P2$ lattice (`p1top2`) expressed in the $P1$ reference frame (`p1`).

In [18]:
T_p1top2_p1 = s1['N'][ct1-1]/s1['D'][ct1-1]
print(T_p1top2_p1)

[[ 0.7281106  -0.46082949  0.29339478]
 [ 0.65437788  1.0921659   0.14132104]
 [-0.48387097  0.12903226  0.7311828 ]]


The **CSL** and the **DSC** lattice are determined using the `find_csl_dsc` function in the `byxtal.find_csl_dsc` module. The lattices are expressed using the basis vectors of the primitive cell $(P1)$. The following varibales are used:
+ `l_csl_p`: The lattice (`l`) of the CSL (`csl`) expressed in the primitive reference frame (`p`).
+ `l_dsc_p`: The lattice (`l`) of the DSC (`dsc`) expressed in the primitive reference frame (`p`).

In [19]:
# l_csl_p, l_dsc_p = fcd.find_csl_dsc(l_p_po, T_p1top2_p1, 1e-6)
# l_p_po = Matrix(l_p_po)
T_p1top2_p1 = np.array(T_p1top2_p1, dtype='double')
TI_p1top2_p1 = T_p1top2_p1*sig_num
TI_p1top2_p1 = Matrix((np.around(TI_p1top2_p1)).astype(int))
l_csl_p = fcd.csl_finder(TI_p1top2_p1, sig_num, l_p_po, 1e-6)
print(l_csl_p)
# csl_mats[sig_id] = l_csl_p; 

AttributeError: 'numpy.ndarray' object has no attribute 'inv'

Now, we will determine the boundary-plane symmetry group (and the axes) for the $\Sigma$ misorientation. The following steps are followed:
+ First, we need to cnvert the rotation in the $P$ reference frame (`T_p1top2_p1`) to $PO$ reference frame (`T_p1top2_po1`).
+ The misorientation is converted to a quaternion using the `mat2quat` function in the `byxtal.tools` module.
+ The disorientation is obtained using the `misorient_fz` function in the `byxtal.misorient_fz` module.
+ The bicrystallography symmetry properties are computed using `disorient_symm_props` function in the `byxtal.disorient_symm_props` module. The outputs are stored in the following variables:
    - `bp_symm_grp`: The point group symmetry corresponding to the given misorientation.
    - `symm_grp_ax`: The axes of the symmetry point group are stored in a $3 \times 3$ matrix. The columns correspond to the $\hat{e}_x$, $\hat{e}_y$, $\hat{e}_z$ axes of the symmetry point group.

In [ ]:
l_p_po = l1.l_p_po
l_po_p = np.linalg.inv(l_p_po)
T_p1top2_po1 = np.dot(l_p_po, np.dot(T_p1top2_p1, l_po_p))
quat1 = bxt.mat2quat(T_p1top2_po1)
dis_quat1 = mfz.misorient_fz(quat1, l1.cryst_ptgrp)
x_g, y_g, z_g, bp_symm_grp = dsp.disorient_symm_props(dis_quat1, l1.cryst_ptgrp)
symm_grp_ax = (np.vstack((x_g, y_g, z_g))).transpose()
print(bp_symm_grp)
print(symm_grp_ax)

One could compute the bycrystallography properties of a large number of $\Sigma$ rotation by using the code snippets discussed above and putting all of them together in a for-loop. This is shown in the code below.

In [13]:
n1 = 1; n2 = 400; sig_nums = 2*np.arange(n1,n2)+1;

num_sigs = 0;
sig_mats = {};
csl_mats = {};
# dsc_mats = {};
csl_bp_props = {};

for sig_num in sig_nums:
    s1 = cuf.csl_rotations(sig_num, sig_type, l1);
    for ct1 in range(np.shape(s1['N'])[0]):
        bp_symm_grp_props = {};
        sig_id = str(sig_num)+'_'+str(ct1+1);
        print(sig_id)
        #### Store the sigma-misorientation (in 'p' reference frame)
        T_p1top2_p1 = s1['N'][ct1]/s1['D'][ct1];
        sig_mats[sig_id] = T_p1top2_p1;

        # l_csl_p, l_dsc_p = fcd.find_csl_dsc(l_p_po, T_p1top2_p1, 1e-6)
        # l_p_po = Matrix(l_p_po)
        T_p1top2_p1 = np.array(T_p1top2_p1, dtype='double')
        TI_p1top2_p1 = T_p1top2_p1*sig_num
        TI_p1top2_p1 = Matrix((np.around(TI_p1top2_p1)).astype(int))
        l_csl_p = fcd.csl_finder(TI_p1top2_p1, sig_num, Matrix(l_p_po), 1e-6)
        csl_mats[sig_id] = l_csl_p; 
        # dsc_mats[sig_id] = l_dsc_p;

        #### Generate boundary-planpe orientations
        l_p_po = l1.l_p_po; l_po_p = np.linalg.inv(l_p_po);
        T_p1top2_po1 = np.dot(l_p_po, np.dot(T_p1top2_p1, l_po_p));

        ## Find the corresponding disorientation
        quat1 = bxt.mat2quat(T_p1top2_po1); print(quat1);
        dis_quat1 = mfz.misorient_fz(quat1, l1.cryst_ptgrp); print(dis_quat1);
        x_g, y_g, z_g, bp_symm_grp = dsp.disorient_symm_props(dis_quat1, l1.cryst_ptgrp)
        symm_grp_ax = (np.vstack((x_g, y_g, z_g))).transpose();
        bp_symm_grp_props['symm_grp_ax'] = symm_grp_ax;
        bp_symm_grp_props['bp_symm_grp'] = bp_symm_grp;
        csl_bp_props[sig_id] = bp_symm_grp_props;

import pickle as pkl;
pkl_name = 'cF_Id_csl_common_rotations.pkl';
csl_props = {};
csl_props['sig_mats'] = sig_mats;
csl_props['csl_mats'] = csl_mats;
# csl_props['dsc_mats'] = dsc_mats;
csl_props['csl_bp_props'] = csl_bp_props;

jar = open(pkl_name, 'wb');
pkl.dump(csl_props, jar);
jar.close()

3_1
[0.8660254  0.28867513 0.28867513 0.28867513 1.        ]
[[0.8660254 ]
 [0.28867513]
 [0.28867513]
 [0.28867513]
 [1.        ]]
5_1
[ 9.48683298e-01  3.16227766e-01  2.92569456e-17 -3.65711820e-17
  1.00000000e+00]
[[ 9.48683298e-01]
 [ 3.16227766e-01]
 [ 2.92569456e-17]
 [-3.65711820e-17]
 [ 1.00000000e+00]]
7_1
[0.94491118 0.18898224 0.18898224 0.18898224 1.        ]
[[0.94491118]
 [0.18898224]
 [0.18898224]
 [0.18898224]
 [1.        ]]
9_1
[ 9.42809042e-01  2.35702260e-01  2.35702260e-01 -1.47196168e-17
  1.00000000e+00]
[[ 9.42809042e-01]
 [ 2.35702260e-01]
 [ 2.35702260e-01]
 [-1.47196168e-17]
 [ 1.00000000e+00]]
11_1
[ 9.04534034e-01  3.01511345e-01  3.01511345e-01 -3.06849434e-17
  1.00000000e+00]
[[ 9.04534034e-01]
 [ 3.01511345e-01]
 [ 3.01511345e-01]
 [-3.06849434e-17]
 [ 1.00000000e+00]]
13_1
[0.97072534 0.13867505 0.13867505 0.13867505 1.        ]
[[0.97072534]
 [0.13867505]
 [0.13867505]
 [0.13867505]
 [1.        ]]
13_2
[ 9.80580676e-01  1.96116135e-01 -5.66104887e-17

57_1
[0.92717265 0.26490647 0.26490647 0.         1.        ]
[[0.92717265]
 [0.26490647]
 [0.26490647]
 [0.        ]
 [1.        ]]
57_2
[0.86094603 0.33113309 0.33113309 0.19867985 1.        ]
[[0.86094603]
 [0.33113309]
 [0.33113309]
 [0.19867985]
 [1.        ]]
57_3
[0.99339927 0.06622662 0.06622662 0.06622662 1.        ]
[[0.99339927]
 [0.06622662]
 [0.06622662]
 [0.06622662]
 [1.        ]]
57_4
[0.93658581 0.28097574 0.18731716 0.09365858 1.        ]
[[0.93658581]
 [0.28097574]
 [0.18731716]
 [0.09365858]
 [1.        ]]
59_1
[0.92057462 0.36822985 0.09205746 0.09205746 1.        ]
[[0.92057462]
 [0.36822985]
 [0.09205746]
 [0.09205746]
 [1.        ]]
59_2
[9.20574618e-01 2.76172385e-01 2.76172385e-01 1.50751363e-17
 1.00000000e+00]
[[9.20574618e-01]
 [2.76172385e-01]
 [2.76172385e-01]
 [1.50751363e-17]
 [1.00000000e+00]]
59_3
[0.97641683 0.19528337 0.06509446 0.06509446 1.        ]
[[0.97641683]
 [0.19528337]
 [0.06509446]
 [0.06509446]
 [1.        ]]
61_1
[9.95893206e-01 9.05357

89_1
[0.9743912  0.22485951 0.         0.         1.        ]
[[0.9743912 ]
 [0.22485951]
 [0.        ]
 [0.        ]
 [1.        ]]
89_2
[0.9743912  0.14990634 0.14990634 0.07495317 1.        ]
[[0.9743912 ]
 [0.14990634]
 [0.14990634]
 [0.07495317]
 [1.        ]]
89_3
[0.9009982  0.37099926 0.15899968 0.15899968 1.        ]
[[0.9009982 ]
 [0.37099926]
 [0.15899968]
 [0.15899968]
 [1.        ]]
89_4
[0.95399809 0.21199958 0.21199958 0.         1.        ]
[[0.95399809]
 [0.21199958]
 [0.21199958]
 [0.        ]
 [1.        ]]
89_5
[0.89943803 0.29981268 0.22485951 0.22485951 1.        ]
[[0.89943803]
 [0.29981268]
 [0.22485951]
 [0.22485951]
 [1.        ]]
91_1
[0.89104211 0.26207121 0.26207121 0.26207121 1.        ]
[[0.89104211]
 [0.26207121]
 [0.26207121]
 [0.26207121]
 [1.        ]]
91_2
[0.89104211 0.36689969 0.26207121 0.05241424 1.        ]
[[0.89104211]
 [0.36689969]
 [0.26207121]
 [0.05241424]
 [1.        ]]
91_3
[ 9.43456353e-01  3.14485451e-01  1.04828484e-01 -7.35475879e-18

[9.40720868e-01 2.82216261e-01 1.88144174e-01 3.68807270e-18
 1.00000000e+00]
[[9.40720868e-01]
 [2.82216261e-01]
 [1.88144174e-01]
 [3.68807270e-18]
 [1.00000000e+00]]
113_2
[0.86474714 0.33259505 0.26607604 0.26607604 1.        ]
[[0.86474714]
 [0.33259505]
 [0.26607604]
 [0.26607604]
 [1.        ]]
113_3
[0.93126615 0.33259505 0.13303802 0.06651901 1.        ]
[[0.93126615]
 [0.33259505]
 [0.13303802]
 [0.06651901]
 [1.        ]]
113_4
[9.97785158e-01 6.65190105e-02 1.39085931e-17 2.78171863e-17
 1.00000000e+00]
[[9.97785158e-01]
 [6.65190105e-02]
 [1.39085931e-17]
 [2.78171863e-17]
 [1.00000000e+00]]
113_5
[0.98775691 0.14110813 0.04703604 0.04703604 1.        ]
[[0.98775691]
 [0.14110813]
 [0.04703604]
 [0.04703604]
 [1.        ]]
115_1
[ 9.89070710e-01  1.31876095e-01  6.59380473e-02 -2.80622764e-17
  1.00000000e+00]
[[ 9.89070710e-01]
 [ 1.31876095e-01]
 [ 6.59380473e-02]
 [-2.80622764e-17]
 [ 1.00000000e+00]]
115_2
[0.97913005 0.13987572 0.13987572 0.04662524 1.        ]
[[0.97

[0.91970901 0.24525574 0.24525574 0.1839418  1.        ]
[[0.91970901]
 [0.24525574]
 [0.24525574]
 [0.1839418 ]
 [1.        ]]
133_7
[0.95382097 0.17342199 0.17342199 0.17342199 1.        ]
[[0.95382097]
 [0.17342199]
 [0.17342199]
 [0.17342199]
 [1.        ]]
135_1
[0.91287093 0.30429031 0.24343225 0.12171612 1.        ]
[[0.91287093]
 [0.30429031]
 [0.24343225]
 [0.12171612]
 [1.        ]]
135_2
[0.90369611 0.30123204 0.30123204 0.04303315 1.        ]
[[0.90369611]
 [0.30123204]
 [0.30123204]
 [0.04303315]
 [1.        ]]
135_3
[0.90369611 0.30123204 0.21516574 0.21516574 1.        ]
[[0.90369611]
 [0.30123204]
 [0.21516574]
 [0.21516574]
 [1.        ]]
135_4
[0.97372899 0.18257419 0.12171612 0.06085806 1.        ]
[[0.97372899]
 [0.18257419]
 [0.12171612]
 [0.06085806]
 [1.        ]]
135_5
[0.98976241 0.12909944 0.04303315 0.04303315 1.        ]
[[0.98976241]
 [0.12909944]
 [0.04303315]
 [0.04303315]
 [1.        ]]
135_6
[0.94672926 0.25819889 0.17213259 0.0860663  1.        ]
[[0.9

...

[0.95556903 0.28104971 0.08431491 0.02810497 1.        ]
[[0.95556903]
 [0.28104971]
 [0.08431491]
 [0.02810497]
 [1.        ]]
633_6
[0.91416793 0.31797145 0.23847859 0.07949286 1.        ]
[[0.91416793]
 [0.31797145]
 [0.23847859]
 [0.07949286]
 [1.        ]]
633_7
[0.97378758 0.17885894 0.09936608 0.09936608 1.        ]
[[0.97378758]
 [0.17885894]
 [0.09936608]
 [0.09936608]
 [1.        ]]
633_8
[0.97378758 0.17885894 0.13911251 0.01987322 1.        ]
[[0.97378758]
 [0.17885894]
 [0.13911251]
 [0.01987322]
 [1.        ]]
633_9
[0.97378758 0.21860537 0.05961965 0.01987322 1.        ]
[[0.97378758]
 [0.21860537]
 [0.05961965]
 [0.01987322]
 [1.        ]]
633_10
[0.92746406 0.36536463 0.05620994 0.05620994 1.        ]
[[0.92746406]
 [0.36536463]
 [0.05620994]
 [0.05620994]
 [1.        ]]
633_11
[0.92746406 0.22483977 0.22483977 0.1967348  1.        ]
[[0.92746406]
 [0.22483977]
 [0.22483977]
 [0.1967348 ]
 [1.        ]]
633_12
[0.8744215  0.31797145 0.27822502 0.23847859 1.        ]
[[

[0.9311039  0.25753938 0.25753938 0.01981072 1.        ]
[[0.9311039 ]
 [0.25753938]
 [0.25753938]
 [0.01981072]
 [1.        ]]
637_21
[0.86851431 0.33619909 0.33619909 0.14008295 1.        ]
[[0.86851431]
 [0.33619909]
 [0.33619909]
 [0.14008295]
 [1.        ]]
637_22
[0.98058068 0.16809954 0.08404977 0.05603318 1.        ]
[[0.98058068]
 [0.16809954]
 [0.08404977]
 [0.05603318]
 [1.        ]]
637_23
[0.91129318 0.23772866 0.23772866 0.23772866 1.        ]
[[0.91129318]
 [0.23772866]
 [0.23772866]
 [0.23772866]
 [1.        ]]
639_1
[0.87030655 0.3560345  0.27691572 0.19779694 1.        ]
[[0.87030655]
 [0.3560345 ]
 [0.27691572]
 [0.19779694]
 [1.        ]]
639_2
[0.89008624 0.3362548  0.21757664 0.21757664 1.        ]
[[0.89008624]
 [0.3362548 ]
 [0.21757664]
 [0.21757664]
 [1.        ]]
639_3
[0.90986594 0.3560345  0.19779694 0.07911878 1.        ]
[[0.90986594]
 [0.3560345 ]
 [0.19779694]
 [0.07911878]
 [1.        ]]
639_4
[0.90986594 0.27691572 0.23735633 0.19779694 1.        ]
[[

[0.92674953 0.25633498 0.25633498 0.09859038 1.        ]
[[0.92674953]
 [0.25633498]
 [0.25633498]
 [0.09859038]
 [1.        ]]
643_21
[0.92674953 0.37464343 0.01971808 0.01971808 1.        ]
[[0.92674953]
 [0.37464343]
 [0.01971808]
 [0.01971808]
 [1.        ]]
643_22
[0.92674953 0.21689883 0.21689883 0.21689883 1.        ]
[[0.92674953]
 [0.21689883]
 [0.21689883]
 [0.21689883]
 [1.        ]]
645_1
[9.74480581e-01 1.94896116e-01 1.11369209e-01 2.84824307e-17
 1.00000000e+00]
[[9.74480581e-01]
 [1.94896116e-01]
 [1.11369209e-01]
 [2.84824307e-17]
 [1.00000000e+00]]
645_2
[ 9.74480581e-01  2.22738419e-01  2.78423023e-02 -1.42412153e-17
  1.00000000e+00]
[[ 9.74480581e-01]
 [ 2.22738419e-01]
 [ 2.78423023e-02]
 [-1.42412153e-17]
 [ 1.00000000e+00]]
645_3
[0.97448058 0.16705381 0.13921151 0.0556846  1.        ]
[[0.97448058]
 [0.16705381]
 [0.13921151]
 [0.0556846 ]
 [1.        ]]
645_4
[0.89095367 0.30626533 0.25058072 0.22273842 1.        ]
[[0.89095367]
 [0.30626533]
 [0.25058072]
 [0

[0.91596018 0.27756369 0.27756369 0.08326911 1.        ]
[[0.91596018]
 [0.27756369]
 [0.27756369]
 [0.08326911]
 [1.        ]]
649_17
[0.88320226 0.29440075 0.29440075 0.21589388 1.        ]
[[0.88320226]
 [0.29440075]
 [0.29440075]
 [0.21589388]
 [1.        ]]
649_18
[ 9.71472918e-01  2.22050953e-01  8.32691072e-02 -7.14265295e-18
  1.00000000e+00]
[[ 9.71472918e-01]
 [ 2.22050953e-01]
 [ 8.32691072e-02]
 [-7.14265295e-18]
 [ 1.00000000e+00]]
649_19
[0.97147292 0.16653821 0.16653821 0.02775637 1.        ]
[[0.97147292]
 [0.16653821]
 [0.16653821]
 [0.02775637]
 [1.        ]]
649_20
[0.88820381 0.33307643 0.30532006 0.08326911 1.        ]
[[0.88820381]
 [0.33307643]
 [0.30532006]
 [0.08326911]
 [1.        ]]
649_21
[0.88820381 0.33307643 0.24980732 0.19429458 1.        ]
[[0.88820381]
 [0.33307643]
 [0.24980732]
 [0.19429458]
 [1.        ]]
649_22
[0.98133584 0.15701373 0.07850687 0.07850687 1.        ]
[[0.98133584]
 [0.15701373]
 [0.07850687]
 [0.07850687]
 [1.        ]]
651_1
[0.99

Exception: Lattice Reduction did not work.

In [ ]:
sig_mats